In [5]:
from core.db_handler import DbHandler

In [6]:
handler = DbHandler(
    '/Users/adamgroszeibl/Documents/airflow_brokerchooser/analytics_pipeline/db/sources',
    '/Users/adamgroszeibl/Documents/airflow_brokerchooser/analytics_pipeline/db/brokerchooser_db.duckdb'
)

handler.create_schemas()
handler.run_db(create_tables=True)

2025-02-23 23:19:24.273 | INFO     | core.db_handler:load_db:211 - Loaded /Users/adamgroszeibl/Documents/airflow_brokerchooser/analytics_pipeline/db/sources/broker_data.csv into staging.broker_data in DuckDB.
2025-02-23 23:19:24.876 | INFO     | core.db_handler:load_db:211 - Loaded /Users/adamgroszeibl/Documents/airflow_brokerchooser/analytics_pipeline/db/sources/brokerchooser_conversions.csv into staging.brokerchooser_conversions in DuckDB.
2025-02-23 23:19:24.885 | INFO     | core.db_handler:load_db:211 - Loaded /Users/adamgroszeibl/Documents/airflow_brokerchooser/analytics_pipeline/db/sources/page_category_mapping.csv into staging.page_category_mapping in DuckDB.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
query = '''
SELECT 
    MAX(conversion_seconds)
FROM analytics.brokerchooser



'''

df = handler.fetch_df(query)

import pandas as pd
pd.set_option('display.max_rows', None)

df



,max(conversion_seconds)
0,6473


In [4]:
import dash
from dash import dcc, html
import plotly.express as px

app = dash.Dash(__name__)

query = '''
SELECT 
    conversion_country
    , COUNT(1) AS count
FROM analytics.brokerchooser
WHERE 
    broker_id IS NOT NULL
    AND important_score > 50
GROUP BY 1
ORDER BY 1 ASC
'''

df = handler.fetch_df(query)

# Create Plotly Figure
fig = px.bar(df, x='conversion_country', y='count',
              title='Conversion Count by Country',)

# Dash Layout
app.layout = html.Div([
    html.H1("Brokerchooser Analytics Dashboard"),
    dcc.Graph(figure=fig)
])

# Run App
if __name__ == '__main__':
    app.run_server(debug=True)

/Users/adamgroszeibl/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from core.analyst import Analyst

analyst = Analyst()

df = analyst.calc_khi_square()

df['is_significant'] = df['p_value'] < 0.05

df

,page_category,conversion_count,total_count,conversion_rate,chi2,p_value,is_significant
0,Scam shield,30.0,87,0.344828,0.306574,5.797901e-01,False
1,Not categorized,1375.0,4161,0.330449,10.281625,1.343616e-03,True
2,Country pages,4004.0,12200,0.328197,29.016210,7.177519e-08,True
3,Broker review - scaled account opening,124.0,385,0.322078,0.418188,5.178427e-01,False
4,Best brokers - cfd,8.0,25,0.320000,0.000000,1.000000e+00,False
5,Best brokers - citizen,4388.0,13721,0.319802,16.661757,4.467256e-05,True
6,Comparison table,531.0,1690,0.314201,0.873426,3.500086e-01,False
7,Broker review - scaled deposits,130.0,414,0.314010,0.171686,6.786180e-01,False
8,Find my broker,3705.0,11989,0.309033,3.066332,7.992966e-02,False
9,How to buy shares,562.0,1838,0.305767,0.157135,6.918076e-01,False


In [18]:
import pandas as pd
import numpy as np

df = pd.read_parquet("/Users/adamgroszeibl/Documents/airflow_brokerchooser/analytics_pipeline/python_db/analytics/analytics_brokerchooser")
df["min_conversion_seconds"] = df.groupby("broker_id")["conversion_seconds"].transform("min")

df["broker_id"] = df["broker_id"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["broker_timestamp"] = df["broker_timestamp"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["broker_country_residency"] = df["broker_country_residency"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["broker_ip_country"] = df["broker_ip_country"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["important_score"] = df["important_score"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["important_score_bucket"] = df["important_score_bucket"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["conversion_seconds"] = df["conversion_seconds"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["is_converted"] = df["is_converted"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)
df["min_conversion_seconds"] = df["min_conversion_seconds"].mask(df["conversion_seconds"] != df["min_conversion_seconds"], np.nan)

df = df.sort_values("conversion_seconds", ascending=False)

df.head(100)

,conversion_id,session_id,session_action_counter,last_action_by_session_count,conversions_by_session,conversion_country,is_mobile,ui_element,created_at,measurement_category,page_category,broker_id,broker_timestamp,broker_country_residency,broker_ip_country,important_score,important_score_bucket,conversion_seconds,is_converted,min_conversion_seconds
8683,5518120,76016925,1.0,1.0,NaN,HONG KONG,True,dynamic score top broker list row,2024-05-07 00:31:07,01bestca/02notrel/03online,Best brokers - citizen,1715041973/Germany,2024-05-07 00:32:53,<NA>,GERMANY,0.0,1 - Below 10,106.0,True,106.0
91586,5454150,75180518,1.0,1.0,NaN,UNITED STATES,True,<NA>,2024-05-01 05:15:24,01review/02notrel/03review,Broker reviews - general,1714540449/Cyprus,2024-05-01 05:14:09,<NA>,CYPRUS,0.0,1 - Below 10,75.0,True,75.0
28529,5573261,76716581,1.0,1.0,NaN,UNITED STATES,True,dynamic score top broker list row,2024-05-12 14:19:25,01bestca/02notrel/03online,Best brokers - citizen,1715523640/United States of America,2024-05-12 14:20:40,<NA>,UNITED STATES OF AMERICA,0.0,1 - Below 10,75.0,True,75.0
80465,5392333,74024093,1.0,1.0,NaN,CAMEROON,True,dynamic score top broker list row,2024-04-24 00:33:55,01bestca/02notrel/03online,Best brokers - citizen,1713918904/Cameroon,2024-04-24 00:35:04,<NA>,CAMEROON,0.0,1 - Below 10,69.0,True,69.0
77415,5377005,73789297,1.0,1.0,NaN,INDONESIA,True,fmb broker list item link,2024-04-22 05:14:46,01fmybro/02notrel/03notrel,Find my broker,1713762822/United States of America,2024-04-22 05:13:42,<NA>,UNITED STATES OF AMERICA,0.0,1 - Below 10,64.0,True,64.0
56186,5608601,77141800,2.0,2.0,NaN,SLOVENIA,True,mobile-chapter-2-top5-box,2024-05-16 05:47:45,01safety/02isscam/03notrel,<NA>,1715838403/Philippines,2024-05-16 05:46:43,<NA>,PHILIPPINES,0.0,1 - Below 10,62.0,True,62.0
64749,5703490,79018493,1.0,1.0,NaN,INDONESIA,True,desktop-top5-box,2024-05-27 06:46:29,01safety/02issafe/03notrel,<NA>,1716792329/United States of America,2024-05-27 06:45:29,<NA>,UNITED STATES OF AMERICA,0.0,1 - Below 10,60.0,True,60.0
84660,5416892,74418221,2.0,2.0,NaN,VIETNAM,True,desktop-top5-box,2024-04-26 15:36:01,01review/02notrel/03stonxx,Broker review - scaled stocks,1714145819/Czechia,2024-04-26 15:36:59,CZECH REPUBLIC,CZECHIA,0.0,1 - Below 10,58.0,True,58.0
1208,5478860,70083098,1.0,2.0,NaN,MALAYSIA,True,dynamic score top broker list row,2024-05-02 06:49:12,01bestca/02notrel/03notrel,Best brokers - general,1714632496/Puerto Rico,2024-05-02 06:48:16,PUERTO RICO,PUERTO RICO,0.0,1 - Below 10,56.0,True,56.0
278,5473692,75243208,1.0,1.0,NaN,CAMBODIA,True,mobile-chapter-2-top5-box,2024-05-01 15:59:37,01bestca/02notrel/03notrel,Best brokers - general,1714579121/Serbia,2024-05-01 15:58:41,<NA>,SERBIA,0.0,1 - Below 10,56.0,True,56.0
